# Model zur Bonitätsanalyse

In diesem Notebook widmen wir uns der klassischen Disziplin von KI in Banken, der Entwicklung von Scoringmodellen zur Bonitätsbewertung. Wir nutzen hierfür einen Datensatz von Kreditkartennutzern und versuchen, den Ausfall bzw. Nichtausfall der Kunden vorherzusagen.

##  Laden der Bibliotheken und Daten

In [ ]:
import os
import bisect

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt

from ipywidgets import interact
import ipywidgets as widgets

pd.set_option("display.max_columns", 50)

In [ ]:
dataset_path = os.environ['DATASET_PATH']
ds = pd.read_csv(dataset_path + '/finance/credit card defaults_train.csv')
ds.head()

In [ ]:
X_train = ds.drop(['id', 'default'], axis=1)
y_train = ds.default

## Erstellung des Modells

### Erster, naiver Versuch

Zuerst trainieren wir einfach mit den Daten ohne jegliche Aufbereitung eine logistische Regression. Dies ergibt eine Fehlermeldung, da nicht alle Felder numerisch sind.

In [ ]:
lm = LogisticRegression()

lm.fit(X_train, y_train)

## Mit korrekten, quantitativen Features

Tatsächlich sind die Felder "education" und "sex" qualitative Felder und können daher nicht vearbeitet werden. Wir transformieren sie daher in Dummy-Spalten. Diese neuen Spalten enthalten je nach Ausprägung eine Null oder eine Eins und sind damit valide quantiative Eingangsgrößen für eine logistische Regression. Die Transformation in Dummy-Werte wird auch One-Hot-Encoding genannt.

In [ ]:
X_train_dummies = pd.get_dummies(X_train, columns=['education', 'sex'], drop_first=True)

Das Ergebnis der Transformation sieht man in den neuen Spalten ganz rechts im Data Frame

In [ ]:
X_train_dummies.head()

Nun können wir eine logistische Regression durchführen und erhalten eine mäßige AUC von 65%.

In [ ]:
lm = LogisticRegression(solver='liblinear')
lm.fit(X_train_dummies, y_train)

_ = plot_roc_curve(lm, X_train_dummies, y_train)

## Mit Binning

Mit einer logistischen Regression können wir nicht-lineare (wie z.B. nicht-monotone oder nicht-gleichmäßige) Einflüsse prinzipienbedingt nicht abbilden. Daher nutzt man oft das sogenannte "Binning", mit dem der Wertebereich einer Variablen in mehrere Intervalle unterteilt wird und der Effekt für die Intervalle separat modelliert wird.

### Alter

Zuerst führen wir ein Binning des Alters durch und beschränken uns dabei vorerst nur auf diese Variable (ein sogenanntes univariates Modell)

#### Univariates Modell für das Alter

An der ROC-Kurve sehen wir, dass der Einfluss des Alters nicht-monoton ist

In [ ]:
lm = LogisticRegression(solver='liblinear')
lm.fit(X_train[['age']], y_train)
_ = plot_roc_curve(lm, X_train[['age']], y_train)

Wir schauen uns den vorhandenen Wertebereich an

In [ ]:
X_train.age.min(), X_train.age.max()

Diesen zerlegen wir mit Stützpunkten in 8 Intervalle: <= 24, 25-29, 30-34, 35-39, 40-49, 50-59, 60-69 und >= 70. Auch diese Intevalle stellen wir über Dummy-Variablen dar.

In [ ]:
age_binned = np.digitize(X_train.age, [25, 30, 35, 40, 50, 60, 70])
age_dummies = pd.get_dummies(age_binned, prefix='age_bin')
age_dummies.head()

In [ ]:
age_dummies.head()

Wir schauen uns die Wirkungsweise noch einmal genauer an, indem wir das Alter links neben die Dummy-Matrix stellen

In [ ]:
pd.concat([X_train.age, age_dummies], axis=1).head()

#### Univariates Modell für das Alter mit Binning

Nun nutzen wir unsere neu gebauten Bins, um unser Modell zu verfeinern. Tatsächlich sehen wir nun einen (kleinen) Effekt mit einer AUC von 53%.

In [ ]:
lm = LogisticRegression(solver='liblinear')
lm.fit(age_dummies, y_train)
_ = plot_roc_curve(lm, age_dummies, y_train)

Den beobachteten nicht-monotonen Zusammenhang können wir nun direkt an den Regressionskoeffizienten ablesen. Wir sehen, dass sowohl geringes wie hohes Alter mit einer höheren Ausfallwahrscheinlichkeit einhergeht. Der kleinste Wert liegt bei Bin Nummer 2, also dem Intevall 30-34.

In [ ]:
_ = plt.plot(lm.coef_[0])

### Payment_delay_0

Ähnlich wie oben verfahren wir nun für die Variable "payment_delay_0"

#### Univariates Modell für payment_delay_0

In [ ]:
lm = LogisticRegression(solver='liblinear')
lm.fit(X_train[['payment_delay_0']], y_train)
_ = plot_roc_curve(lm, X_train[['payment_delay_0']], y_train)

#### Jetzt mit Dummies

In [ ]:
X_train.payment_delay_0.value_counts()

In [ ]:
pd0_dummies = pd.get_dummies(X_train.payment_delay_0, prefix='pd0_dummy')

In [ ]:
pd0_dummies.head()

#### Univariates Modell für payment_delay_0 mit Dummies

Auch hier sehen wir durch das Binning eine (leichte) Verbesserung der AUC

In [ ]:
lm = LogisticRegression(solver='liblinear')
lm.fit(pd0_dummies, y_train)
_ = plot_roc_curve(lm, pd0_dummies, y_train)

In [ ]:
_ = plt.plot(lm.coef_[0])

## Zusammenstellen der Features

Nun stellen wir die neuen durch das Binning erhalten Features mit den anderen Features zusammen

In [ ]:
sex_dummies = pd.get_dummies(X_train.sex, prefix='sex_dummy')
education_dummies = pd.get_dummies(X_train.education, prefix='education_dummy')

In [ ]:
X_features = pd.concat([sex_dummies, education_dummies, age_dummies, pd0_dummies], axis=1)

In [ ]:
X_features.head()

## Training des Modells

Wir trainieren das neue Modell und sehen eine deutliche Verbesserung der AUC auf nun 73% (gegenüber 65% ohne Binning)

In [ ]:
lm = LogisticRegression(solver='liblinear')
lm.fit(X_features, y_train)
_ = plot_roc_curve(lm, X_features, y_train)

## Validierung auf Testdaten

Wir überprüfen nun, ob das Modell auch auf den unabhängigen Testdaten ähnlich gute Qualität hat

In [ ]:
ds_test = pd.read_csv(dataset_path + '/finance/credit card defaults_test.csv')
ds_test.head()

Entsprechend müssen wir nun sämtliche Schritte der Vorverarbeitung, inklusive des Binning, auf diesen Daten nachexzerzieren. In einem weniger prototypischen Code würden wir diese Verarbeitung in Funktionen kapseln und diese Redundanz vermeiden.

In [ ]:
X_test = ds_test.drop(['id', 'default'], axis=1)
y_test = ds_test.default

In [ ]:
X_test_dummies = pd.get_dummies(X_test, columns=['education', 'sex'], drop_first=True)

In [ ]:
age_binned_test = np.digitize(X_test.age, [25, 30, 35, 40, 50, 60, 70])
age_dummies_test = pd.get_dummies(age_binned_test, prefix='age_bin')

In [ ]:
pd0_dummies_test = pd.get_dummies(X_test.payment_delay_0, prefix='pd0_dummy')

In [ ]:
sex_dummies_test = pd.get_dummies(X_test.sex, prefix='sex_dummy')
education_dummies_test = pd.get_dummies(X_test.education, prefix='education_dummy')

In [ ]:
X_features_test = pd.concat([sex_dummies_test, education_dummies_test, age_dummies_test, pd0_dummies_test], axis=1)

Wir sehen, dass die AUC mit 73% gleich hoch ist wie auf den Trainingsdaten. Es liegt also kein Overfitting vor und das Modell wurde damit erfolgreich auf den Testdaten validiert.

In [ ]:
_ = plot_roc_curve(lm, X_features_test, y_test)

### Interaktive ROC-Kurve

Mit der folgenden interaktiven Grafik kann die Funktionsweise der AUC nachvollzogen werden. Durch den Regler verändert man den Schwellwert, der zwischen Ausfällen und Nicht-Ausfällen trennt. Mit niedrigerem Schwellwert hat man mehr korrekt positive, aber auch mehr falsch positive Vorhersagen. Wenn man all diese Punktepaare aus korrekt positiven und falsch positiven Anteilen aufträgt, erhält man die Receiver Operating Characteristic (ROC). Die Fläche unter dieser Kurve ist die Area Under Curve (AUC).

In [ ]:
pred = lm.predict_proba(X_features_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, pred)
thresholds_rev = list(reversed(thresholds))
roc_auc = roc_auc_score(y_test, pred)

In [ ]:
def roc_with_threshold(thr):
    i = max(bisect.bisect_right(thresholds_rev, thr), 1)
    i = len(thresholds_rev) - i

    pred_discrete = pred > thr
    cm = np.pad(confusion_matrix(y_test, pred_discrete), (0, 1))
    cm[2] = cm.sum(axis=0)
    cm[:, 2] = cm.sum(axis=1)
    dfcm = pd.DataFrame(
        cm,
        index=pd.MultiIndex.from_product([['Actual'], ['Negative', 'Positive', 'Sum']]),
        columns=pd.MultiIndex.from_product([['Model'], ['Negative', 'Positive', 'Sum']])
    )
    dfcm['Model', 'Positive rate'] = dfcm['Model', 'Positive'] / dfcm['Model', 'Sum']

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.scatter(fpr[i], tpr[i], c='red', s=40)
    plt.show()

    display(dfcm)

In [ ]:
style = {'description_width': '150px'}
layout = widgets.Layout(width='400px')

_ = interact(
    roc_with_threshold,
    thr=widgets.SelectionSlider(
        options=np.round(np.linspace(1, 0, 101), 2),
        description='Threshold',
        layout=layout,
        style=style,
        orientation='horizontal',
        readout=True
    )
)

## Vergleich mit Random Forest

Die logistische Regression ist ein etabliertes und leicht erklärbares Verfahren, kann jedoch in vielen Fällen nicht mit leistungsfähigeren Verfahren wie z.B. einem Random Forest mithalten. Hier erproben wir exemplarisch, wie sich ein Random Forest auf den Daten schlägt.

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_dummies, y_train)

Man sieht, dass der Random Forest auf den Trainingsdaten ein optimales Ergebnis erzielt. Dies ist für diese Modellklasse üblich.

In [ ]:
_ = plot_roc_curve(rf, X_train_dummies, y_train)

Entscheidend ist daher die Qualität auf den Testdaten. Hier erreicht der Random Forest eine AUC von 77% und ist damit ein gutes Stück besser als die logistiche Regression.

In [ ]:
_ = plot_roc_curve(rf, X_test_dummies, y_test)

## Vergleich mit Gradient Boosted Trees

Ein oft noch leistungsfähigeres Verfahren sind die sogenannten Gradient Boosted Trees

In [ ]:
gbt = GradientBoostingClassifier()
gbt.fit(X_train_dummies, y_train)

Ähnlich wie ein Random Forest lernen sie die Trainingsdaten auswändig. Auch hier ist also die Qualität auf den Testdaten entscheidend.

In [ ]:
_ = plot_roc_curve(gbt, X_train_dummies, y_train)

Mit 79% AUC ist diese tatsächlich noch etwas besser als beim Random Forest

In [ ]:
_ = plot_roc_curve(gbt, X_test_dummies, y_test)